In [1]:
import importlib
import gui
import people
import astar
import main_functions
importlib.reload(gui)
importlib.reload(people)
importlib.reload(astar)
importlib.reload(main_functions)
from gui import GUI
from people import PeopleList
from main_functions import group_num_calculator,one_simulation
import matplotlib.pyplot as plt

# Test Model

In [13]:
# [(start_boundary), (end_boundary), (door), (start_obstacle1), (end_obstacle1), (start_obstacle2), (end_obstacle2)...]
# Note the boader of room is of size 1, hence the real room is (1,1)->(26,16)
barrier_set = [(0, 0), (27, 17), (26, 8), (20, 4), (23, 7), (20, 10), (23, 13)]

seed = 41
delta_time = 0.005  # Timestrp
percent_threshold = 50  # Checkpoint: the percentage of people being moved out

initial_people_count = None
recorded_time = False

# Build GUI
gui = GUI()
gui.add_barrier(barrier_set)
gui.update_gui()

# Build PeopleList saving each people object
people_list = PeopleList(seed)
# Build direction matrix
people_list.direction_matrix(barrier_set)
# Assign groups
r_1 = 0.6 # ratio of 1-ped group
r_2 = 0.35 # ratio of 2-ped group
r_3 = 0.05 # ratio of 3-ped group
N = 15*9 # Total pedestrians
(n_1,n_2,n_3) = group_num_calculator(N, r_1, r_2, r_3)
group_split = {3: n_3, 2: n_2}
people_list.assign_groups(group_split,seed)


time = 0

# Initial each people
for people in people_list.list:
    gui.add_line(people, people_list.list, people.group_id)
    gui.add_oval(people.loc[0] - people.r, people.loc[1] - people.r,
                 people.loc[0] + people.r, people.loc[1] + people.r, people.id)
gui.update_gui()

initial_people_count = len(people_list.list)

# Move
# start_time = real_time.time() 

while people_list.list:
    i = 0
    while i < len(people_list.list):
        gui.del_line(people_list.list[i].group_id)
        gui.del_oval(people_list.list[i].id)

        if people_list.list[i].loc[0] > 1040 and people_list.list[i].loc[1] >300 and people_list.list[i].loc[1] <380:  # delete people if it goes out of room
            people_list.list.pop(i)
            continue
        i += 1

    # Check the percentage of people who have moved out
    remaining_people_count = len(people_list.list)
    moved_out_percentage = ((initial_people_count - remaining_people_count) / initial_people_count) * 100

    if not recorded_time and moved_out_percentage >= percent_threshold:
        print(f"Time when {percent_threshold}% of people moved out: {round(time, 3)} seconds") 
        recorded_time = True

    people_list.move(barrier_set, delta_time=delta_time,  A_p=2000, A_o=5000, B=-0.08, threshold_ped=1.2, threshold_obs=120,
                     threshold_group=2,q_A=1, beta1=0.4, beta2=0.2)  # ped movement

    for people in people_list.list:  # update position
        gui.add_oval(int(people.loc[0]) - people.r,
                     int(people.loc[1]) - people.r, int(people.loc[0]) + people.r,
                     int(people.loc[1]) + people.r, people.id)
        gui.add_line(people, people_list.list, people.group_id)

    # time = real_time.time() - start_time  # update time
    time += delta_time
    gui.update_time(str(round(time, 3)))
    gui.update_gui()

gui.start()


Time when 50% of people moved out: 24.035 seconds


In [ ]:
r_1 = 0.6 # ratio of 1-ped group
r_2 = 0.35 # ratio of 2-ped group
r_3 = 0.05 # ratio of 3-ped group
N = 15*9 # Total pedestrians
barrier_set = [(0, 0), (27, 17), (26, 8), (20, 4), (23, 7), (20, 10), (23, 13)]
delta_time = 0.005
seed = 41
percent_threshold = 80
A_p=2000
A_o=5000
B=-0.08
beta1=0.4
beta2=0.2
time = one_simulation(barrier_set, percent_threshold, 
                   r_1, r_2, r_3, A_p, A_o, B, beta1, beta2,
                   N, delta_time, seed, gui_display=True)

In [14]:
print(f"Time when 100% of people moved out: {time:.3f} seconds")

Time when 100% of people moved out: 57.735 seconds


# Global Sensitive Analysis

# Local Sensitive Analysis

# Experiment

In [2]:
r_1 = 0.6 # ratio of 1-ped group
r_2 = 0.35 # ratio of 2-ped group
r_3 = 0.05 # ratio of 3-ped group
N = 15*9 # Total pedestrians
barrier_set = [(0, 0), (27, 17), (26, 8), (20, 4), (23, 7), (20, 10), (23, 13)]
delta_time = 0.005
seed = 41
percent_threshold = 80
A_p=2000
A_o=5000
B=-0.08
beta1=0.4
beta2=0.2
time = one_simulation(barrier_set, percent_threshold, 
                   r_1, r_2, r_3, A_p, A_o, B, beta1, beta2,
                   N, delta_time, seed, gui_display=False)

Time when 80% of people moved out: 41.79 seconds


In [3]:
print(f"Time when 100% of people moved out: {time:.3f} seconds")

Time when 100% of people moved out: 57.735 seconds
